# NAME

> Fill in a module description here

In [2]:
# | default_exp feature_view

In [1]:
# | export
from __future__ import annotations
from typing import Dict, List, Optional, Union
from dataclasses import dataclass, field
from datetime import datetime
from snowflake.snowpark import DataFrame
from snowflake.ml.feature_store import FeatureView, Entity

from snowflake_feature_store.core import create_version, create_feature_view_name


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# | export
@dataclass
class FeatureViewConfig:
    """Configuration for feature views
    
    Args:
        name: Base name for the feature view
        domain: Business domain (e.g., 'RETAIL', 'FINANCE')
        entity: Main entity name (e.g., 'CUSTOMER', 'PRODUCT')
        feature_type: Type of features (e.g., 'BEHAVIOR', 'PROFILE')
        major_version: Major version number
        minor_version: Minor version number
        refresh_frequency: How often to refresh features
        timestamp_col: Column name for temporal features
        description: Optional description of the feature view
        feature_descriptions: Descriptions of individual features
        selected_features: Optional list of features to include
    """
    name: str
    domain: str = ""
    entity: str = "CUSTOMER"
    feature_type: str = "BASE"
    major_version: int = 1
    minor_version: int = 0
    refresh_frequency: str = "1 day"
    timestamp_col: Optional[str] = None
    description: Optional[str] = None
    feature_descriptions: Optional[Dict[str, str]] = None
    selected_features: Optional[List[str]] = None
    tags: Dict[str, str] = field(default_factory=dict)
    
    def __post_init__(self):
        "Set derived attributes after initialization"
        self.full_name = create_feature_view_name(
            self.domain, self.entity, self.feature_type)
        self.version = create_version(self.major_version, self.minor_version)
        
        # Add default description if none provided
        if not self.description:
            self.description = f"Feature view {self.name}"
            
        # Add creation timestamp to tags
        self.tags['created_at'] = datetime.utcnow().isoformat()
        self.tags['feature_type'] = self.feature_type
        if self.domain:
            self.tags['domain'] = self.domain

In [4]:
# | export
class FeatureViewBuilder:
    """Builder for creating feature views
    
    Args:
        config: Feature view configuration
        feature_df: DataFrame containing feature transformations
        entities: List of entities associated with features
    """
    def __init__(self, 
                 config: FeatureViewConfig,
                 feature_df: DataFrame,
                 entities: Union[Entity, List[Entity]]):
        self.config = config
        self.feature_df = feature_df
        self.entities = [entities] if isinstance(entities, Entity) else entities
        
    def build(self) -> FeatureView:
        """Build the feature view
        
        Returns:
            Configured FeatureView object ready for registration
        """
        # Create base feature view
        feature_view = FeatureView(
            name=self.config.full_name,
            entities=self.entities,
            feature_df=self.feature_df,
            refresh_freq=self.config.refresh_frequency,
            timestamp_col=self.config.timestamp_col,
            desc=self.config.description
        )
        
        # Add feature descriptions if provided
        if self.config.feature_descriptions:
            feature_view = feature_view.attach_feature_desc(
                self.config.feature_descriptions
            )
            
        # Apply feature selection if specified
        if self.config.selected_features:
            feature_view = feature_view.slice(self.config.selected_features)
            
        return feature_view


In [5]:
# | hide
# Test setup
from snowflake_feature_store.connection import get_connection
conn = get_connection()
session = conn.session


In [6]:
# | hide
def test_feature_view_config():
    "Test feature view configuration"
    config = FeatureViewConfig(
        name="customer_behavior",
        domain="RETAIL",
        entity="CUSTOMER",
        feature_type="BEHAVIOR",
        feature_descriptions={
            "AVG_SESSION_LENGTH": "Average session length in minutes",
            "TOTAL_PURCHASES": "Total number of purchases"
        }
    )
    
    # Test name generation
    assert config.full_name == "FV_RETAIL_CUSTOMER_BEHAVIOR"
    
    # Test version generation
    assert config.version == "V1_0"
    
    # Test tags
    assert 'feature_type' in config.tags
    assert config.tags['domain'] == 'RETAIL'
    assert 'created_at' in config.tags

def test_feature_view_builder():
    "Test feature view builder"
    # Create test data
    df = session.create_dataframe([
        ['C1', 10, 100],
        ['C2', 20, 200]
    ], ['customer_id', 'session_length', 'purchases'])
    
    # Create test entity
    entity = Entity(
        name="CUSTOMER",
        join_keys=["customer_id"],
        desc="Customer entity"
    )
    
    # Create config
    config = FeatureViewConfig(
        name="customer_behavior",
        domain="RETAIL",
        feature_descriptions={
            "session_length": "Session length in minutes",
            "purchases": "Number of purchases"
        }
    )
    
    # Build feature view
    builder = FeatureViewBuilder(config, df, entity)
    feature_view = builder.build()
    
    # Verify feature view properties
    assert feature_view.name == config.full_name
    assert len(feature_view.entities) == 1
    assert feature_view.entities[0].name == "CUSTOMER"

# Run tests
test_feature_view_config()
test_feature_view_builder()

In [10]:
# | export
def create_feature_view(
    config: FeatureViewConfig,
    feature_df: DataFrame,
    entities: Union[Entity, List[Entity]]
) -> FeatureView:
    """Convenience function to create a feature view
    
    Args:
        config: Feature view configuration
        feature_df: DataFrame containing feature transformations
        entities: Entity or list of entities
        
    Returns:
        Configured FeatureView object
        
    Example:
        >>> config = FeatureViewConfig(
        ...     name="customer_behavior",
        ...     domain="RETAIL",
        ...     feature_descriptions={"session_length": "Session length in minutes"}
        ... )
        >>> entity = Entity("CUSTOMER", ["customer_id"])
        >>> feature_view = create_feature_view(config, df, entity)
    """
    return FeatureViewBuilder(config, feature_df, entities).build()


In [11]:
#| hide
import nbdev; nbdev.nbdev_export()

In [12]:

# | eval: false
# Example usage
from snowflake_feature_store.feature_view import *

# Create test data
df = session.create_dataframe([
    ['C1', 10, 100],
    ['C2', 20, 200]
], ['customer_id', 'session_length', 'purchases'])

# Create entity
customer_entity = Entity(
    name="CUSTOMER",
    join_keys=["customer_id"],
    desc="Customer entity"
)

# Create feature view config
config = FeatureViewConfig(
    name="customer_behavior",
    domain="RETAIL",
    feature_type="BEHAVIOR",
    feature_descriptions={
        "session_length": "Session length in minutes",
        "purchases": "Number of purchases"
    }
)

# Create feature view
feature_view = create_feature_view(config, df, customer_entity)

print(f"Created feature view: {feature_view.name}")
print(f"With features: {feature_view.feature_df.columns}")


Created feature view: FV_RETAIL_CUSTOMER_BEHAVIOR
With features: ['CUSTOMER_ID', 'SESSION_LENGTH', 'PURCHASES']
